In [75]:
#! pip install PyAudio
#! pip install SpeechRecognition
import pandas as pd
from pycorenlp import StanfordCoreNLP
#!pip install azure-cognitiveservices-speech
#!pip install google-api-python-client 
#!pip install --upgrade google-cloud-speech
#!pip install anvil-uplink
#!pip install --upgrade google-api-python-client
#!pip install --upgrade google-cloud-speech

from azure.cognitiveservices.speech import AudioDataStream, SpeechConfig, SpeechSynthesizer, SpeechSynthesisOutputFormat
from azure.cognitiveservices.speech.audio import AudioOutputConfig
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import anvil.server
anvil.server.connect("XBQM36YNDUZ4TWI522HM4KDL-NUKYRRNENRSFFQ4A")
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/HP/Downloads/bionic-hallway-326706-923256f9f8eb.json"

In [76]:
class isQuestionBasic():
    def __init__(self):
        self.nlp = StanfordCoreNLP('http://localhost:9000')
    def isQuestion(self, sentence):
        if '?' in sentence:
            return 1
        output = self.nlp.annotate(sentence, properties={
            'annotators': 'parse',
            'outputFormat': 'json',
            'timeout': 1000,
        })
        if ('SQ' or 'SBARQ') in output['sentences'][0]["parse"]:
            return 1    
        else:
            return 0

In [77]:
def silience(file_name):
    from pydub import AudioSegment,silence
    myaudio = intro = AudioSegment.from_wav(file_name)
    silence = silence.detect_silence(myaudio, min_silence_len=1000, silence_thresh=-16)
    silence = [((start/1000),(stop/1000)) for start,stop in silence] #convert to sec
    return silence

In [78]:
def Question_detection(transcript):
    obj = isQuestionBasic()
    #transcript is a list of setences ex. ["hello everyone","how is everyone","lets start todays class"]
    

In [79]:
def delete_blob(blob_name, bucket_name):
    """Deletes a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"

    storage_client = storage.Client.from_service_account_json(
        'C:/Users/HP/Downloads/bionic-hallway-326706-923256f9f8eb.json')

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

    print("Blob {} deleted.".format(blob_name))

In [80]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file, bucket_name):
    """ Upload data to a bucket"""

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'C:/Users/HP/Downloads/bionic-hallway-326706-923256f9f8eb.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [81]:
# Imports the Google Cloud client library
def speech_to_text(file_path):
    
    from google.cloud import speech

    upload_to_bucket("MeetingOWL",file_path,"nlp_j")
    # Instantiates a client
    client = speech.SpeechClient()

    # The name of the audio file to transcribe
    gcs_uri = "gs://nlp_j/machine-learning_speech-recognition_30-4447-0004.wav"

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",enable_word_time_offsets=True,enable_automatic_punctuation=True,
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)
    #print(response)
    word_stamps=[]
    transcript=[]
    
    for result in response.results:
        alternative = result.alternatives[0]
        for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time
            word_stamps.append([word,start_time,end_time])
        transcript.append(alternative.transcript)
    delete_blob("MeetingOWL","nlp_j")
    return [transcript,word_stamps]
   # return response.results 
#speech_to_text("finally.mp3")

In [86]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [83]:
stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [84]:
def ask_question(question,all_data,k,threshold):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)[0]
    sim=[]
    for i in range(len(similarity)):
        sim.append((similarity[i],i))
    sim.sort(reverse=True)
    #use the top k results 
    iteresting,difficult = 0,0

    for max_indices in range(k):
        print(all_data.iloc[sim[max_indices][1]]['Question'],all_data.iloc[sim[max_indices][1]]['Is the question difficult?'])
        max_index=sim[max_indices][1]
        max_sim = sim[max_indices][0]
        weight_diff = -1
        weight_interesting = -1
        if(all_data.iloc[max_index]['Is the question difficult?']=='Yes'): weight_diff=1
        if(all_data.iloc[max_index]['Is the question interesting?']=='Yes'): weight_interesting=1
        difficult+= weight_diff*max_sim
        iteresting+= weight_interesting*max_sim
    print('Your question:', question)
    print('Is the Question difficult:',end=' ')
    
    if(difficult>=threshold):
        print("Yes")
    else:
        print("No")
    print('Is the Question interesting?:',end=' ')
    if(iteresting>=threshold):
        print("Yes")
    else:
        print("No")
    

In [ ]:
import anvil.media

In [ ]:
def write(cendel):
    try:
        anvil.media.download(cendel)
    except AssertionError as error:
        return error

In [ ]:
import anvil.media
obj = isQuestionBasic()
@anvil.server.callable
def classify_question(question):
    return write(question)

In [ ]:
import anvil.media
from shutil import copy
@anvil.server.callable
def write_media_to_file(media_object):
    with anvil.media.TempFile(media_object) as f:
        #audio file stored in finally.mp3
        copy(f, "finally.mp3")
        #convert the speech to text
        transcript,word_stamps=speech_to_text("finally.mp3")
        #response_time()
        #detect_questions()
        #interesting_questions()
        return "Success"

        

In [ ]:
#path to the form responses
df = pd.read_csv("Desktop/NLP_J/question_feedback.csv")


In [ ]:
df.iloc[0:,1:]
df=df.rename(columns={'Write a question relevant to our subjects which profs ask(eg. What is NLP?, Can someone tell me the difference between ML and AI? etc.)':'Question'})

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(df['Question']))
#print(tfidf_matrix.shape)

In [ ]:
# ask_question('What is 2+2',df,3,0)
# ask_question('What is the main purpose of OS',df,1,0)